# Code Description<br>

This code is a code that collects patent PDFs using a web crawler for patent network analysis research.<br>
Data is collected through a web page called KIPRIS that provides Korean patent data.<br>
The data collected are the patent application PDF itself and various information about the patent.<br>

Written By Sangmin Hwang(terran3125@u.sogang.ac.kr)

### 1. Import Libraries & Set Working Directory

In [ ]:
#!pip install selenium
#!pip install tqdm

In [1]:
# Basic Libraries
import os, time
import pandas as pd
import numpy as np
from tqdm import trange, tqdm

# Web Scraping Methods
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchWindowException 

Set Directory to download csv files which have information about Patents

In [2]:
#os.chdir(r'C:\Users\tkdal\OneDrive - Sogang\sogang_pypr\기술금융 데이터\KIPRIS_csv') # 노트북
os.chdir(r'C:\Users\tkdals\OneDrive - Sogang\sogang_pypr\기술금융 데이터\KIPRIS_csv') # 연구실

### 2. Set Chrome Driver Options

In [3]:
options = webdriver.ChromeOptions()
#options.add_argument('headless') # Don't show Chrome Driver Working
options.add_argument("disable-gpu")
options.add_argument("lang=ko_KR") # Language is Korean
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36") # Visit Website like a human
options.add_experimental_option("prefs", {
  "download.default_directory": r"D:\KIPRIS_RawData(2022.11.20)", # Where to download raw PDF files(very large files.. Recommend set D-drive which have N-TB)
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
})

In [4]:
# 크롬 드라이버 열기
#driver = webdriver.Chrome(r'C:\Users\tkdal\Dropbox\기술금융\키프리스 크롤링\chromedriver.exe', chrome_options=options) # 노트북 경로
driver = webdriver.Chrome(r'C:\Users\tkdals\OneDrive - Sogang\TechnologyFinancingDataCleansing\ChromeDriver\chromedriver.exe', chrome_options=options)
#driver = webdriver.Chrome(r'C:\Users\pc조교\Desktop\pypr\chromedriver.exe', chrome_options=options) # 연구실 경로

C:\Users\tkdals\AppData\Local\Temp\ipykernel_23156\817290157.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\tkdals\OneDrive - Sogang\TechnologyFinancingDataCleansing\ChromeDriver\chromedriver.exe', chrome_options=options)
C:\Users\tkdals\AppData\Local\Temp\ipykernel_23156\817290157.py:3: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(r'C:\Users\tkdals\OneDrive - Sogang\TechnologyFinancingDataCleansing\ChromeDriver\chromedriver.exe', chrome_options=options)


### 3. Enter KIPRIS web page & Set patent search criteria<br>
### __Code Reference__<br>
Crawling Target: http://kportal.kipris.or.kr/kportal/search/search_patent.do<br>
Selenium Basic Usage: https://workingwithpython.com/howtouseselenium-2/<br>
Properties of selenium elements: https://minorman.tistory.com/113<br>
Change Active Tabs: https://devyurim.github.io/python/crawler/2018/08/13/crawler-3.html<br>
Xpath: https://wkdtjsgur100.github.io/selenium-xpath/<br>
By method description: https://blog.daum.net/sualchi/13721870

In [5]:
# Enter KIPRIS
driver.get('http://kpat.kipris.or.kr/kpat/searchLogina.do?next=MainSearch')

In [6]:
# Clear all text in the search box
# 검색창에 글씨 있으면 다 지우고
driver.find_element(By.NAME, 'queryText').clear()

In [7]:
# Enter 'Display' in the search box.
# 검색창에 디스플레이를 입력한다.
driver.find_element(By.NAME, 'queryText').send_keys('디스플레이')

In [8]:
# 좌측 행정상태 전부 해제
xpath_button = '//*[@id="leftHangjung00"]'
driver.find_element(By.XPATH, xpath_button).click()

In [9]:
# 필요한 행정상태만 체크
xpath_button = '//*[@id="leftHangjung01"]' # 공개
driver.find_element(By.XPATH, xpath_button).click()
xpath_button = '//*[@id="leftHangjung03"]' # 소멸
driver.find_element(By.XPATH, xpath_button).click()
xpath_button = '//*[@id="leftHangjung07"]' # 등록
driver.find_element(By.XPATH, xpath_button).click()

In [10]:
# 불필요한 권리 종류인 '실용' 체크 해제
xpath_button = '//*[@id="leftGubn01"]' # 특허, 실용 중 실용은 필요없으므로 해제 대상
driver.find_element(By.XPATH, xpath_button).click()

In [11]:
# 검색 버튼 클릭하기기
xpath_button = '//*[@id="SearchPara"]/fieldset/span[1]/a/img' # 검색 버튼 xpath 추출
driver.find_element(By.XPATH, xpath_button).click()

In [12]:
# 드롭다운 버튼 중에서 선택
select = Select(driver.find_element(By.ID, 'opt28')) # 페이지당 보여지는 검색결과 갯수 선택
select.select_by_value("90") # 최대 개수인 90개를 한 페이지에 보이게 하자

In [13]:
# 90개 보이기 버튼 클릭
xpath_button = '//*[@id="pageSel"]/a/img' # 90개 리스트 보여주기 go 버튼 클릭
driver.find_element(By.XPATH, xpath_button).click()

In [17]:
# 2005년 특허만
xpath_button = '//*[@id="snavigation"]/ul/li[2]/button/img' # 검색결과 분류통계
driver.find_element(By.XPATH, xpath_button).click()

ElementClickInterceptedException: Message: element click intercepted: Element <img src="/kpat/images/common/btn_category.gif" alt="검색결과 분류통계"> is not clickable at point (450, 742). Other element would receive the click: <div id="PIContents" style="">...</div>
  (Session info: chrome=107.0.5304.107)
Stacktrace:
Backtrace:
	Ordinal0 [0x0026ACD3+2075859]
	Ordinal0 [0x001FEE61+1633889]
	Ordinal0 [0x000FB7BD+571325]
	Ordinal0 [0x00131499+791705]
	Ordinal0 [0x0012F4AC+783532]
	Ordinal0 [0x0012D0AB+774315]
	Ordinal0 [0x0012BD37+769335]
	Ordinal0 [0x00121C76+728182]
	Ordinal0 [0x0014731C+881436]
	Ordinal0 [0x001215BF+726463]
	Ordinal0 [0x00147534+881972]
	Ordinal0 [0x0015B56A+963946]
	Ordinal0 [0x00147136+880950]
	Ordinal0 [0x0011FEFD+720637]
	Ordinal0 [0x00120F3F+724799]
	GetHandleVerifier [0x0051EED2+2769538]
	GetHandleVerifier [0x00510D95+2711877]
	GetHandleVerifier [0x002FA03A+521194]
	GetHandleVerifier [0x002F8DA0+516432]
	Ordinal0 [0x0020682C+1665068]
	Ordinal0 [0x0020B128+1683752]
	Ordi

In [18]:
time.sleep(0.5)
xpath_button = '//*[@id="CategoryList_1"]/li[10]/button' # 2005년
driver.find_element(By.XPATH, xpath_button).click()

In [19]:
# 지금까지 설정한 조건들로 검색된 특허 개수 확인
total_patent_number = int(driver.find_element(By.CLASS_NAME, 'total').text.replace(",",""))
print("조건에 맞는 총 특허 개수: ", total_patent_number)

조건에 맞는 총 특허 개수:  20770


In [20]:
xpath_total_page_number = '/html/body/div[2]/div/section[3]/section/div[2]/form/section/div[1]/p'
total_page_number = int(driver.find_element(By.XPATH, xpath_total_page_number).text.split("/ ")[1].split(" Pages")[0].replace(",",""))
print("디스플레이 검색어 총 페이지 수(페이지당 90개 노출): ", total_page_number)

디스플레이 검색어 총 페이지 수(페이지당 90개 노출):  231


### 4. Open a tab containing patent information

In [21]:
# 첫번째 특허 눌러서 탭 연다.
first = '/html/body/div[2]/div/section[3]/section/div[2]/form/section/article[1]/div[1]/h1/a[2]'
driver.find_element(By.XPATH, first).click()

In [22]:
# Switch the selenium work window to the Patent Information tab
driver.switch_to.window(driver.window_handles[1]) # Switch to an open window which is opend later

In [23]:
def go_to_next(page) :
    while 1 :
        time.sleep(1)
        tab_current_page = int(driver.find_element(By.TAG_NAME,"strong").text) # 새탭에서 현재 페이지
        try :
            time.sleep(1)
            if tab_current_page == page : # 원하는 페이지 입력(단, 5로 나눴을 때 나머지가 반드시 1이어야 함)
                break
            else :
                driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a.next').click()  
                
        except :
            print('실패. 재시도 중')
            time.sleep(5)
            go_to_next(page)


In [24]:
# 다음 버튼 처음 1번은 수동으로 눌러줘야 작동됨 ㅠ
#go_to_next(1)

In [25]:
# 현재 페이지 확인
tab_current_page = int(driver.find_element(By.TAG_NAME,"strong").text) # 새탭에서 현재 페이지
tab_current_page

1

### 5. Download patent PDF & generate patent information CSV

In [28]:
# iframe 나오기
driver.switch_to.default_content()

In [29]:
# iframe 들어가기
#content = driver.find_element(By.TAG_NAME,"iframe")
#driver.switch_to.frame(content) # iframe 안으로 들어간다.

In [30]:
tab_current_page = int(driver.find_element(By.TAG_NAME,"strong").text) # 새탭에서 현재 페이지

In [31]:
#driver.switch_to.window(driver.window_handles[1]) # 새탭으로 창전환

In [32]:
start_page = tab_current_page # 고정된 시작페이지
print("시작페이지 재확인: ", start_page)

patent_link = []
error_patent_link = []

시작페이지 재확인:  1


In [33]:
# 변경하는 코드

def collect_patent(start_page, end_page) :

    #for page in trange(1, 5) :
    #for page in trange(1, total_page_number) :
    for page in trange(start_page, end_page+1) : 
        time.sleep(5)
        tab_current_page = int(driver.find_element(By.TAG_NAME,"strong").text) # 새탭에서 현재 페이지

        for article in range(1, 91) : # 한 페이지 내의 1~90번째 특허를 보면서
            
            # 변수 초기화
            PatentName_Kor='';PatentName_Eng='';IPC='';CPC='';
            Application_No='';Application_Date='';Applicant='';Public_No='';
            Public_Date='';Legal_State='';Category='';Claim_Count=''; Family_Count = '';
            Expiration_Date=''; Summary_Text=''; Claim_Text='';

            where_error=''
            try :                
                xpath_patent = '/html/body/div[1]/section/div/div[2]/div/ul/li[%d]/a' %article # 특허 순서대로 선택하기
                driver.find_element(By.XPATH, xpath_patent).click()
                time.sleep(1)
                
                where_error='PatentName'
                PatentName_Kor = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/section[1]/div/h1/span[1]').text
                PatentName_Eng = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/section[1]/div/h1/span[2]').text              
                    
                content = driver.find_element(By.TAG_NAME,"iframe")
                driver.switch_to.frame(content) # iframe 안으로 들어간다.
                
                tmNews = driver.find_element(By.CLASS_NAME, 'detial_plan_info')
                tmNewsLis = tmNews.find_elements(By.TAG_NAME, 'li') # div 내에서 li 리스트 검색

                where_error='서지정보내부'                          
                for li in tmNewsLis :
                    
                    if  li.text[:12] == '(51) Int. CL' :
                        try : IPC = li.text.split('(51) Int. CL')[1]
                        except : continue
                    elif  li.text[:8] == '(52) CPC' :
                        try : CPC = li.text.split('(52) CPC')[1]
                        except : continue
                    elif  li.text[:16] == '(21) 출원번호/일자(국제)' :
                        try :
                            Application_No = li.text.split('(21) 출원번호/일자(국제) ')[1].split(" ")[0]
                            Application_Date = li.text.split('(21) 출원번호/일자(국제) ')[1].split(" ")[1][1:-1]
                        except : continue
                    elif  li.text[:13] == '(21) 출원번호/일자 ' :
                        try :
                            Application_No = li.text.split('(21) 출원번호/일자 ')[1].split(" ")[0]
                            Application_Date = li.text.split('(21) 출원번호/일자 ')[1].split(" ")[1][1:-1]
                        except : continue                                                
                    elif  li.text[:8] == '(71) 출원인' :
                        try : Applicant = li.text.split('(71) 출원인 ')[1]
                        except : continue
                    elif li.text[:8] == '번역문제출일자 ':
                        try : translation_issue_date = li.text.split('번역문제출일자 ')[1][1:-1]
                        except : continue
                    elif li.text[:12] == '(65) 공개번호/일자' :
                        try:
                            Public_No = li.text.split('(65) 공개번호/일자 ')[1].split(" ")[0]
                            Public_Date = li.text.split('(65) 공개번호/일자 ')[1].split(" ")[1][1:-1]
                        except : continue
                    elif li.text[:5] == '법적상태 ' :
                        try : Legal_State = li.text.split('법적상태 ')[1]
                        except : continue
                    elif li.text[:6] == '심사청구항수' :
                        try: 
                            Claim_Count = li.text.split('심사청구항수 ')[1]
                        except : continue
                    elif li.text[:3] == '구분 ' :
                        try : Category = li.text.split('구분 ')[1]
                        except : continue                              
                where_error='요약문 텍스트'
                Summary_Text=driver.find_element(By.XPATH, '//*[@id="abstract_box"]/summary').text 
                # pdf 다운로드
                #pdfDownload = driver.find_element(By.CLASS_NAME, 'right_align')
                #pdfDownload2 = pdfDownload.find_element(By.TAG_NAME,"a")
                #pdfDownload2.click()
                time.sleep(1)   
                driver.switch_to.default_content() # iframe에서 나온다. 이거 무조건 해줘야 한다.   


                # 이제 청구항 텍스트 받으러 갈 거다.
                where_error='청구항 텍스트'
                driver.find_element(By.XPATH, '//*[@id="liViewSub04"]/a').click() # 상세정보 中 청구항 클릭

                content = driver.find_element(By.TAG_NAME,"iframe")
                driver.switch_to.frame(content) # iframe 안으로 들어간다.

                tBody= driver.find_element(By.CLASS_NAME, 'tstyle_list')
                trLis = tBody.find_elements(By.TAG_NAME, 'tr') # tBody 내에서 lr 리스트 검색
                
                for i in range(1,len(trLis)) :
                    if trLis[i].text[1:].replace(" ", "")=='삭제' :
                        continue
                    Claim_Text = Claim_Text + trLis[i].text[1:].replace("\n", "")

                driver.switch_to.default_content() # iframe에서 나온다. 이거 무조건 해줘야 한다. 
                time.sleep(1)


                where_error='패밀리 개수'
                # 이제 패밀리 개수 받으러 갈거야
                driver.find_element(By.XPATH, '/html/body/div[1]/div/div/section[1]/div/ul/li[1]/ul/li[7]/a').click()
                
                content = driver.find_element(By.TAG_NAME,"iframe")
                driver.switch_to.frame(content) # iframe 안으로 들어간다.
             
                tBody= driver.find_element(By.CLASS_NAME, 'tstyle_list')
                trLis = tBody.find_elements(By.TAG_NAME, 'tr') # tBody 내에서 lr 리스트 검색
                try:
                    if trLis[1].text == '데이터가 존재하지 않습니다.' : # 이렇게 떠 있으면 Family 개수 0
                        Family_Count = 0
                    else :
                        Family_Count = len(trLis)-1 # 1개 이상 있으면 이 규칙으로 패밀리 개수 Count
                except:
                    Family_Count = 0 # 데이터가 존재하지 않는다는 문구도 없을 수도 있다.

                driver.switch_to.default_content()
                time.sleep(1)
                
                
                # 소멸한 특허면 등록사항에 소멸일자 받으러 갈 거야
                where_error='등록사항'
                if Legal_State[:2] == '소멸' :      
                    driver.find_element(By.CSS_SELECTOR, '#liView07 > a').click()   
                    time.sleep(0.5)

                    content = driver.find_element(By.TAG_NAME,"iframe")
                    driver.switch_to.frame(content) # iframe 안으로 들어간다.
                    
                    time.sleep(0.5)

                    tBody= driver.find_element(By.XPATH, '/html/body/table[1]')
                    trLis = tBody.find_elements(By.TAG_NAME, 'tr') # tBody 내에서 tr 리스트 검색
                    
                    time.sleep(0.5)

                    for tr in trLis :
                        if tr.text[:6] == '등록원인일자' :
                            try : Expiration_Date = tr.text.split("등록원인일자 : ")[1].split(" 등 록")[0]
                            except : continue                    
                                  
                    driver.switch_to.default_content()
                    time.sleep(1)
                    
                    
                                              
                patent_info = {
                    #'Page_No' : tab_current_page,
                    'Application_No' : Application_No,
                    'Application_Date' : Application_Date,
                    'Applicant' : Applicant,
                    'Legal_State' : Legal_State,
                    'PatentName_Kor' : PatentName_Kor,
                    'PatentName_Eng' : PatentName_Eng,
                    'IPC' : IPC,
                    'CPC' : CPC, 
                    #'Public_No' : Public_No,
                    #'Public_Date' : Public_Date,
                    #'Category' : Category,
                    'Claim_Count' : Claim_Count,
                    'Family_Count' : Family_Count,
                    'Expiration_Date' : Expiration_Date,
                    'Summary_Text' : Summary_Text,
                    'Claim_Text' : Claim_Text
                }
                patent_link.append(patent_info)        

                
            except :
                print('에러발생! 페이지, 출원번호, 에러발생요소: ',tab_current_page, Application_No, where_error)
                driver.switch_to.window(driver.window_handles[-1])
                
                error_patent_info = {
                    'Page_No' : tab_current_page,
                    'Application_No' : Application_No,
                    'WhereError' :where_error
                }
                error_patent_link.append(error_patent_info)
                continue
    
        if tab_current_page == end_page : # end_page는 5의 배수니까 
            time.sleep(7)
            driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a.next').click()
            tab_current_page = int(driver.find_element(By.TAG_NAME,"strong").text) # 새탭에서 현재 페이지
            print("스크래핑 종료")
            break                     
        elif (tab_current_page % 5) == 1 :
            time.sleep(7)
            #driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a:nth-child(3)').click()
            driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a:nth-child(3)').send_keys(Keys.ENTER)
        elif (tab_current_page % 5) == 2 :
            time.sleep(7)
            #driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a:nth-child(4)').click() 
            driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a:nth-child(4)').send_keys(Keys.ENTER)
        elif (tab_current_page % 5) == 3 :
            time.sleep(7)
            #driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a:nth-child(5)').click()
            driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a:nth-child(5)').send_keys(Keys.ENTER)
        elif (tab_current_page % 5) == 4 :
            time.sleep(7)
            #driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a:nth-child(6)').click() 
            driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a:nth-child(6)').send_keys(Keys.ENTER) 
        elif (tab_current_page % 5) == 0 :
            time.sleep(7)
            #driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a.next').click()   
            try : driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a.next').send_keys(Keys.ENTER)     
            except :        
                time.sleep(20)
                driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a.next').send_keys(Keys.ENTER) 

    df = pd.DataFrame(patent_link)
    df.to_csv('Normal(%d_%d).csv'%(start_page, end_page), index=False, encoding='utf-8')
    error_df = pd.DataFrame(error_patent_link)
    error_df.to_csv('Error(%d_%d).csv'%(start_page, end_page), index=False, encoding='utf-8')

In [36]:
tab_current_page = int(driver.find_element(By.TAG_NAME,"strong").text) # 새탭에서 현재 페이지
tab_current_page

6

In [37]:
collect_patent(tab_current_page, 231)

  1%|          | 2/226 [24:35<45:53:45, 737.62s/it]

에러발생! 페이지, 출원번호, 에러발생요소:  8 1020050002430 요약문 텍스트


  2%|▏         | 4/226 [48:43<44:57:03, 728.93s/it]

에러발생! 페이지, 출원번호, 에러발생요소:  10 1020050024502 요약문 텍스트


  4%|▍         | 10/226 [2:01:05<43:55:06, 731.97s/it]

...

에러발생! 페이지, 출원번호, 에러발생요소:  30 1020050011728 패밀리 개수


 12%|█▏        | 26/226 [5:16:02<39:45:11, 715.56s/it]

에러발생! 페이지, 출원번호, 에러발생요소:  32 1020077013645 요약문 텍스트
에러발생! 페이지, 출원번호, 에러발생요소:  32 1020077006453 요약문 텍스트
에러발생! 페이지, 출원번호, 에러발생요소:  32 1020077015461 요약문 텍스트


 12%|█▏        | 28/226 [5:35:47<35:57:24, 653.76s/it]

에러발생! 페이지, 출원번호, 에러발생요소:  34 1020050023182 패밀리 개수


# 이하 테스트 코드

In [ ]:
# 새탭 반영한 코드

def collect_patent(start_page, end_page) :
    
    global tab_current_page
    global s_page

    #for page in trange(1, 5) :
    #for page in trange(1, total_page_number) :
    for page in trange(start_page, end_page+1) : 
        time.sleep(5)
        tab_current_page = int(driver.find_element(By.TAG_NAME,"strong").text) # 새탭에서 현재 페이지

        for article in range(1, 91) : # 한 페이지 내의 1~90번째 특허를 보면서
            
            # 변수 초기화
            

            patent_name_Kor='';patent_name_Eng='';IPC='';CPC='';
            Application_No='';Application_Date='';Applicant='';Public_No='';
            Public_Date='';legal_state='';category='';claims_count=''; family_count = '';
            Expiration_Date='';

            try :                
                xpath_patent = '/html/body/div[1]/section/div/div[2]/div/ul/li[%d]/a' %article # 특허 순서대로 선택하기
                driver.find_element(By.XPATH, xpath_patent).click()
                time.sleep(1)
                
                patent_name_Kor = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/section[1]/div/h1/span[1]').text
                patent_name_Eng = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/section[1]/div/h1/span[2]').text              
                    
                content = driver.find_element(By.TAG_NAME,"iframe")
                driver.switch_to.frame(content) # iframe 안으로 들어간다.
                
                tmNews = driver.find_element(By.CLASS_NAME, 'detial_plan_info')
                tmNewsLis = tmNews.find_elements(By.TAG_NAME, 'li') # div 내에서 li 리스트 검색
                                
                for li in tmNewsLis :
                    if  li.text[:12] == '(51) Int. CL' :
                        try : IPC = li.text.split('(51) Int. CL')[1]
                        except : continue
                    elif  li.text[:8] == '(52) CPC' :
                        try : CPC = li.text.split('(52) CPC')[1]
                        except : continue
                    elif  li.text[:16] == '(21) 출원번호/일자(국제)' :
                        try :
                            Application_No = li.text.split('(21) 출원번호/일자(국제) ')[1].split(" ")[0]
                            Application_Date = li.text.split('(21) 출원번호/일자(국제) ')[1].split(" ")[1][1:-1]
                        except : continue
                          
                    elif  li.text[:13] == '(21) 출원번호/일자 ' :
                        try :
                            Application_No = li.text.split('(21) 출원번호/일자 ')[1].split(" ")[0]
                            Application_Date = li.text.split('(21) 출원번호/일자 ')[1].split(" ")[1][1:-1]
                        except : continue                            
                                                    
                    elif  li.text[:8] == '(71) 출원인' :
                        try : Applicant = li.text.split('(71) 출원인 ')[1]
                        except : continue
                    elif li.text[:8] == '번역문제출일자 ':
                        try : translation_issue_date = li.text.split('번역문제출일자 ')[1][1:-1]
                        except : continue
                    elif li.text[:12] == '(65) 공개번호/일자' :
                        try:
                            Public_No = li.text.split('(65) 공개번호/일자 ')[1].split(" ")[0]
                            Public_Date = li.text.split('(65) 공개번호/일자 ')[1].split(" ")[1][1:-1]
                        except : continue
                    elif li.text[:5] == '법적상태 ' :
                        try : legal_state = li.text.split('법적상태 ')[1]
                        except : continue
                    elif li.text[:6] == '심사청구항수' :
                        try: 
                            claims_count = li.text.split('심사청구항수 ')[1]
                        except : continue
                    elif li.text[:3] == '구분 ' :
                        try : category = li.text.split('구분 ')[1]
                        except : continue                              

                # pdf 다운로드
                #pdfDownload = driver.find_element(By.CLASS_NAME, 'right_align')
                #pdfDownload2 = pdfDownload.find_element(By.TAG_NAME,"a")
                #pdfDownload2.click()
                #time.sleep(1)   

                driver.switch_to.default_content() # iframe에서 나온다. 이거 무조건 해줘야 한다.   
                
                # 이제 패밀리 개수 받으러 갈거야
                driver.find_element(By.XPATH, '/html/body/div[1]/div/div/section[1]/div/ul/li[1]/ul/li[7]/a').click()
                
                content = driver.find_element(By.TAG_NAME,"iframe")
                driver.switch_to.frame(content) # iframe 안으로 들어간다.
             
                tBody= driver.find_element(By.CLASS_NAME, 'tstyle_list')
                trLis = tBody.find_elements(By.TAG_NAME, 'tr') # tBody 내에서 lr 리스트 검색
                if trLis[1].text == '데이터가 존재하지 않습니다.' :
                    family_count = len(trLis)-2
                else :
                    family_count = len(trLis)-1

                driver.switch_to.default_content()
                time.sleep(1)
                
                
                # 소멸한 특허면 등록사항에 소멸일자 받으러 갈 거야
                
                if legal_state[:2] == '소멸' :      
                    driver.find_element(By.CSS_SELECTOR, '#liView07 > a').click()   
                    time.sleep(0.5)

                    content = driver.find_element(By.TAG_NAME,"iframe")
                    driver.switch_to.frame(content) # iframe 안으로 들어간다.
                    
                    time.sleep(0.5)

                    tBody= driver.find_element(By.XPATH, '/html/body/table[1]')
                    trLis = tBody.find_elements(By.TAG_NAME, 'tr') # tBody 내에서 tr 리스트 검색
                    
                    time.sleep(0.5)

                    for tr in trLis :
                        if tr.text[:6] == '등록원인일자' :
                            try : Expiration_Date = tr.text.split("등록원인일자 : ")[1].split(" 등 록")[0]
                            except : continue                    
                                  
                    driver.switch_to.default_content()
                    time.sleep(1)
                    
                    
                                              
                patent_info = {
                    'Page_No' : tab_current_page,
                    'Application_No' : Application_No,
                    'Application_Date' : Application_Date,
                    'Applicant' : Applicant,
                    'legal_state' : legal_state,
                    'patent_name_Kor' : patent_name_Kor,
                    'patent_name_Eng' : patent_name_Eng,
                    'IPC' : IPC,
                    'CPC' : CPC, 
                    'Public_No' : Public_No,
                    'Public_Date' : Public_Date,
                    'category' : category,
                    'claims_count' : claims_count,
                    'family_count' : family_count,
                    'Expiration_Date' : Expiration_Date
                }
                patent_link.append(patent_info)        

                
            except :
                print('에러발생!!!   페이지, 출원번호(일단 건너뜀): ',tab_current_page, Application_No)
                driver.switch_to.window(driver.window_handles[-1])
                
                error_patent_info = {
                    'Page_No' : tab_current_page,
                    'Application_No' : Application_No,
                }
                error_patent_link.append(error_patent_info)
                continue
    
        if tab_current_page == end_page : # end_page는 5의 배수니까 
            time.sleep(7)
            driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a.next').click()
            tab_current_page = int(driver.find_element(By.TAG_NAME,"strong").text) # 새탭에서 현재 페이지
            print("스크래핑 종료")
            break                     
        elif (tab_current_page % 5) == 1 :
            time.sleep(7)
            #driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a:nth-child(3)').click()
            driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a:nth-child(3)').send_keys(Keys.ENTER)
        elif (tab_current_page % 5) == 2 :
            time.sleep(7)
            #driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a:nth-child(4)').click() 
            driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a:nth-child(4)').send_keys(Keys.ENTER)
        elif (tab_current_page % 5) == 3 :
            time.sleep(7)
            #driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a:nth-child(5)').click()
            driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a:nth-child(5)').send_keys(Keys.ENTER)
        elif (tab_current_page % 5) == 4 :
            time.sleep(7)
            #driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a:nth-child(6)').click() 
            driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a:nth-child(6)').send_keys(Keys.ENTER) 
        elif (tab_current_page % 5) == 0 :
            time.sleep(7)
            #driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a.next').click()   
            try : driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a.next').send_keys(Keys.ENTER)     
            except :        
                time.sleep(20)
                driver.find_element(By.CSS_SELECTOR, '#divBiblioLeft > div.board_pager02 > a.next').send_keys(Keys.ENTER) 

    df = pd.DataFrame(patent_link)
    df.head()
    df.to_csv('df_%d_%d.csv'%(s_page, end_page), index=False, encoding='utf-8-sig')
 
    error_df = pd.DataFrame(error_patent_link)
    error_df.head()
    error_df.to_csv('error_df_%d_%d.csv'%(s_page, end_page), index=False, encoding='utf-8-sig')